## WORKING WITH TEXT DATA ##

Pada bab ini kita akan mempelajari cara melakukan tokenisasi pada teks  
1. Menyiapkan teks untuk melatih LLM
2. Membagi teks menjadi kata atau token subkata
3. Byte Pair Encoding (BPE)
4. Konversi token menjadi vektor untuk input pada LLM

Akan mencoba membuat tokenizer dengan melatihnya pada dataset the-verdict.txt

#### 2.2 Tokenizing Text

In [1]:
# reading data
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

print('Total karakter : ', len(raw_text))
print(raw_text[:99])

Total karakter :  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Goals kita adalah melakukan tokenisasi terhadap 20479 karakter menjadi per-kata dan spesial karakter, yang kemudian dapat kita lakukan embeddding untuk melatih LLM

Sebagai contoh kita bisa menggunankan library `re` atau `regular expression` untuk split kalimat menjadi bagian per-kata

In [3]:
import re
text = 'Hello, World. This, is a test'
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'World.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test']


In [4]:
# modify the re split on whitespace (\s) 
# semua tanda baca akan dianggap sebagai token individual
result = re.split(r'([,.]|s)', text)
print(result)

['Hello', ',', ' World', '.', ' Thi', 's', '', ',', ' i', 's', ' a te', 's', 't']


Pada saat membuat tokenizer, terkadang hal-hal kecil seperti spasi dan tanda baca akan sangat dibutuhkan dalam memahi maksud dan struktur dari kalimat. Sehingga kita harus benar-benar membuat setiap karakter menjadi token individu

In [5]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()] 
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


Mengunakan `re` untuk melakukan tokenisasi pada data `raw_text` kita 

In [6]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) 
preprocessed = [item.strip() for item in preprocessed if item.strip()] # strip digunakan untuk menghilangkan spasi kosong
print(len(preprocessed))

4690


In [7]:
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


#### 2.3 Converting token into token IDs

Tahap selanjutnya adalah mengkonversi token dari Python string menjadi sebuah angka (int) representatif untuk membuat `token ids`  
Pertama kita membuat sebuah vocabulary. Vocabulary ini akan menjadikan setiap kata unik dan spesial karakter menjadi integer

In [8]:
# sort them alphabetically
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print('Total kata unik : ', vocab_size)

Total kata unik :  1130


In [9]:
# creating vocabulary
vocab = {token: integer for integer, token in enumerate(all_words)}

for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 20:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)


Goals kita selanjutnya adalah menerapkan vocab ini untuk mengkonversi teks baru menjadi `token_ids`

Tokenizer Class  
1. Encode method. Dimana akan memecah teks menjadi token, dan mengubah string menjadi integer untuk membuat `token_ids`  
2. Decode method. Kebalikan dari Encode, decode akan me-reverse dari integer menjadi string yaitu mengubah `token_ids` menjadi teks

In [10]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()} # inverse  

    def encode(self, text): # process text into token_ids
        '''
        pertama - cleaning text
        kedua - menghapus whitespace
        ketiga - mengubah setiap token menjadi token_ids 
        '''
        preprocessed = re.split(r'([,..?_!"()\']|--|\s)', text) 
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids): # convert token_ids back into text
        '''
        pertama - mengubah token_ids menjadi token
        kedua - membersihkan string
        '''
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\']|--)', r'\1', text)
        return text

In [11]:
# try the tokenizer
tokenizer = SimpleTokenizerV1(vocab)
text = """"It's the last he painted, you know," 
       Mrs. Gisburn said with pardonable pride.""" 
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [12]:
# inverse token_ids back to text
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [13]:
# text = "Hello, do you like tea?" 
# print(tokenizer.encode(text))

Text diatas ketika dilakukan encode menjadi token_ids akan mengalami eror, karena kata 'Hello' tidak ditemukan di vocab yang sudah kita buat

#### 2.4 Adding special context tokens

Agar mesin dapat memahami data dengan baik, kita memerlukan tokenizer yang baik yang dapat menangani kata yang tidak diketahui. 

Selanjutnya kita akan membuat tokenizer baru `SimpleTokenizerV2` yang merupakan modifikasi dari tokenizer kita sebelumnya.  
- <|UNK|> : untuk unknown words (kata yang tidak ada di dalam vocabulary)
- <|ENDOFTEXT|> : untuk membagi 2 teks yang tidak berkorelasi

In [14]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}

In [15]:
print(len(vocab.items()))

1132


In [16]:
# lets check
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [17]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) 
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int 
                        else "<|unk|>" for item in preprocessed] # if word not in vocab, replace with <|unk|>
        ids = [self.str_to_int[s] for s in preprocessed] 
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])

        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)   
        return text

In [18]:
text1 = 'Hello, do you like tea?'
text2 = 'In the sunlit terraces of the palace.'

text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [19]:
tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


Terlihat ada token_ids 1131 dan 1130

In [20]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


#### 2.5 Byte pair encoding

BPE atau Byte Pair Encoding adalah teknik tokenisasi yang banyak digunakan di model LLMs seperti GPT-2, GPT-3, dan model asli ChatGPT.  
Karena membangun BPE secara scratch di python sangak kompleks, maka kita akan menggunakan open-source library yaitu `Tiktoken`

In [21]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.9.0


In [22]:
# initialize tiktoken
tokenizer = tiktoken.get_encoding('gpt2')

In [23]:
text = ('Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.')

integers = tokenizer.encode(text, allowed_special={'<|endoftext|>'})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [24]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


**BPE** bekerja dengan memecah kata-kata yang tidak ada di daalam vocabulary menjadi unit subkata yang lebih kecil atau bahkan karakter individu, hal inilah yang memungkingan **BPE** dapat menangani kata-kata diluar vocabulary.

In [25]:
# exercise 
# with word "Akwirw er"

text_exc = 'Akwirw ier'

integers_exc = tokenizer.encode(text_exc)

print(integers_exc)

[33901, 86, 343, 86, 220, 959]


In [26]:
strings_exc = tokenizer.decode(integers_exc)
print(strings_exc)

Akwirw ier


#### 2.6 Data sampling with sliding window

Diberikan sebuah kalimat, model akan mempelajari kalimat dengan memprediksi tiap satu kata setelahnya dan terus berjalan hingga akhir kalimat. 

In [27]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [28]:
# menggunakan hanya 50 sampel pertama saja
enc_sample = enc_text[50:]

In [29]:
context_size = 4 # how many tokens are included in the input

x = enc_sample[:context_size]

y = enc_sample[1:context_size+1]

print(f"x : {x}")
print(f"y : {y}")

x : [290, 4920, 2241, 287]
y : [4920, 2241, 287, 257]


Sederhanya seperti contoh diatas! Sekarang mencoba untuk mengaplikasikannya pada sebuah dataset

In [30]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


inverse to text

In [31]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


Proses diatas menggunakan *stride*  
Token : [10, 45, 22, 110, 88, 300, 50, 120, 95]  
Panjang Potongan (max_length): 4 token  
Stride: 2 token  
Berikut adalah bagaimana potongan-potongan token akan dibuat:

[10, 45, 22, 110]
[22, 110, 88, 300]
[88, 300, 50, 120]
[50, 120, 95]

Sebelum masuk ke **Token Embeddings** ada satu tahap : membuat `data loader` yang mengiterasi seluruh input teks/dataset dan mengembalikan input dan target dalam bentuk tensor PyTorch, dan berbentuk multidimensional array.

In [32]:
import torch 
from torch.utils.data import Dataset, DataLoader

In [33]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt) # tokenizer the entire text

        # proses looping dengan sliding windows
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

        # return total data
    def __len__(self):
        return len(self.input_ids)
        
        # return data
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

Membagi urutan token yang panjang menjadi pasangan input-target. Setiap pasangan input-target adalah potongan token dengan panjang `max_length`. Potongan target adalah potongan input yang digeser satu posisi ke depan, stride menentukan seberapa banyak potongan tumpang tindih. Hasilnya adalah dua daftar, self.input_ids dan self.target_ids, yang berisi tensor PyTorch dari potongan input dan target, masing-masing. 

Membuat sebuah fungsi untuk data loader meng-generate input batch

In [34]:
def create_data_loader_v1(txt, batch_size = 4, max_length = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding('gpt2') # initialize tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride) # create dataset
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

`DataLoader` akan membagi data menjadi batch-batch kecil, alih-alih memproses seluruh dataset secara langsung

In [35]:
# test the GPTDatasetV1 with DataLoader
# with context size of 4, stride 1, and batch_size 1
dataloader = create_data_loader_v1(
    raw_text,
    batch_size=1,
    max_length=4,
    stride=1,
    shuffle=False
)

data_iter = iter(dataloader) # convert dataloader into Python iterator to fetch the next entry via next() function
first_batch = next(data_iter) # get the first batch
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


`first_batch` variabel memuat 2 tensors : pertama - memuat input token_ids, kedua - memuat memuat target token_ids.  
Karena `max_length` = 4, maka setiap tensor terdiri dari 4 token_ids. 

In [36]:
# to more understand about stride = 1
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [37]:
# mmm...lets try to decode
just_try = tokenizer.decode(first_batch[1][0].tolist())
print(just_try)

 HAD always thought


Mari lihat dalam bentuk tensor

In [38]:
dataloader = create_data_loader_v1(
    raw_text,
    batch_size=8,
    max_length=4,
    stride=4,
    shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print('Inputs : \n', inputs)
print('Targets : \n', targets)

Inputs : 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets : 
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


#### 2.7 Create token embedding

Tahap terakhir untuk input teks pada LLM adalah mengkonversi token_ids menjadi embedding vector. Embedding weight diinisialisasi dengan random value (akan dioptimasi dikemudian tahap)

In [39]:
input_ids = torch.tensor([2, 3, 5, 1])

In [40]:
# suppose we have small vocabulary, only 6 words
# and we want to create embedding size of 3
vocab_size = 6
embedding_dim = 3

In [41]:
torch.manual_seed(123) # for reproducibility
embedding_layer = torch.nn.Embedding(vocab_size, embedding_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


Matriks bobot memiliki enam baris dan tiga kolom. Ada satu baris untuk masing-masing dari enam token yang mungkin dalam kosakata, dan ada satu kolom untuk masing-masing dari tiga dimensi penyisipan.

In [42]:
# try in our input_ids previously
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


#### 2.8 Encoding word positions

In [43]:
vocab_size = 50257
output_dim = 256
token_embdding_layer = torch.nn.Embedding(vocab_size, output_dim)

Kita meng-embed setiap token di setiap batch menjadi 256-dimensional vector. Jika kita punya 9 batch, dengan 4 token, maka hasilnya akan 8 x 4 x 256 tensor

In [45]:
max_length = 4
dataloader = create_data_loader_v1(
    raw_text,
    batch_size=8,
    max_length=max_length,
    stride=max_length,
    shuffle=False
)

data_iter = iter(dataloader)

inputs, targets = next(data_iter)

print("Token IDs :\n", inputs)
print("\nInputs shape :", inputs.shape)

Token IDs :
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape : torch.Size([8, 4])


Dapat dilihat `token_ids` tenosr berukuran 8 x 4 dimensi, ini berarti pada satu batch terdapat 8 sampel teks dan 4 token

NB : Pada NLP model seringkali mengeluarkan/menerima tensors dengan ukuran `(batch_size, max_length, output_dim)`.  
- `batch_size` : adalah banyaknya sampel yang akan diproses secara paralel, alih-alih memproses satu per-satu. Contoh, 16 batch size berarti ada 16 sampel teks/urutan teks yang akan diproses
- `max_length` : menunjukkan panjang maksimum dari urutan token dalam batch. Contoh, max length-nya adalah 100 artinya setiap dari 16 sampel diatas memiliki panjang maksimum 100 token
- `output_dim` : setiap token dari setiap sampel tersebut, akan memiliki representasi vektor dengan dimensi yang ditentukan oleh output_dim

In [46]:
# use the embed layer into 256-dimensional vectors
token_embeddings = token_embdding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


Untuk model GPT yang absolut, kita hanya perlu membuat lapisan embedding yang memiliki dimensi embedding yang sama

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length)) # arange from 0 to context_length
print(pos_embedding.shape)

torch.Size([4, 256])


`context_length` adalah variabel yang mewakili ukuran input (token). Pada contoh ini `context_length` memiliki ukuran yang sama dengan panjang maksimum teks (max_length)

Dapat kita lihat dari output, positional-embedding vector merupakan 4 x 256 dimensional. Setelahnya kita bisa langsung menambahkan dengan token embedding, dimana PyTorch akan menambahkan 4 x 256-dimensional `pos_emebedding` dengan 4 x 256-dimensional `token_embeddings` pada setiap batch.

In [48]:
input_embeddings = token_embeddings + pos_embedding
print(input_embeddings.shape)

torch.Size([8, 4, 256])


Pada akhirnya kita telah membuat `input_embeddings` yang sekarang dapat diproses atau dimasukkan ke dalam model LLMs